<a href="https://colab.research.google.com/github/strawndri/tmcc-voculary/blob/feature%2Freconhecimento-texto/reconhecimento-texto/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Instalações e importações

## 1.1 Bibliotecas

In [1]:
!pip install opencv-python
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install langdetect

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [2]:
import pytesseract as pt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from langdetect import detect

In [3]:
!mkdir tessdata

mkdir: cannot create directory ‘tessdata’: File exists


In [4]:
!wget -O ./tessdata/por.traineddata https://github.com/tesseract-ocr/tessdata/blob/main/por.traineddata?raw=true
!wget -O ./tessdata/eng.traineddata https://github.com/tesseract-ocr/tessdata/blob/main/eng.traineddata?raw=true

--2023-08-27 21:59:25--  https://github.com/tesseract-ocr/tessdata/blob/main/por.traineddata?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/tesseract-ocr/tessdata/raw/main/por.traineddata [following]
--2023-08-27 21:59:26--  https://github.com/tesseract-ocr/tessdata/raw/main/por.traineddata
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tesseract-ocr/tessdata/main/por.traineddata [following]
--2023-08-27 21:59:26--  https://raw.githubusercontent.com/tesseract-ocr/tessdata/main/por.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awa

## 1.2 Imagens

In [5]:
artigo = 'imagens/artigo.png'
codigo = 'imagens/codigo.png'
nota_fiscal = 'imagens/nota_fiscal.png'

# 2 Tratamento de imagens

In [6]:
def realcar_detalhes(imagem):
    img_redimensionada = cv2.resize(imagem, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
    img_suavizada = cv2.GaussianBlur(img_redimensionada, (5, 5), 0)
    img_detalhes = 12 * cv2.subtract(img_redimensionada, img_suavizada)

    return cv2.add(img_redimensionada, img_detalhes)

In [7]:
def ajustar_brilho(imagem, threshold=128, alpha=1.5, beta=50):
    if np.mean(imagem) < threshold:
        return cv2.convertScaleAbs(imagem, alpha=alpha, beta=beta)
    return imagem

In [8]:
def binarizar(imagem):
    _, img_threshold = cv2.threshold(imagem, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    if cv2.countNonZero(img_threshold) < (img_threshold.size / 2):
        return 255 - img_threshold
    return img_threshold

# 3 Detecção e extração de textos

In [9]:
def determinar_psm(imagem):
    pixels_pretos = imagem.size - cv2.countNonZero(imagem)
    proporcao_preto = pixels_pretos / imagem.size

    if proporcao_preto > 0.2:
        return '6'
    else:
        return '1'

In [10]:
def detectar_idioma(imagem):
    codigos = {
        'en': 'eng',
        'es': 'spa',
        'fr': 'fra',
        'id': 'ind',
        'it': 'ita',
        'ja': 'jpn',
        'pt': 'por',
        'th': 'tha',
        'uk': 'ukr',
        'zh-cn': 'chi_sim',
        'zh-tw': 'chi_tra'
    }

    try:
        cod_idioma = codigos[detect(pt.image_to_string(imagem))]
    except:
        cod_idioma = 'eng'

    return cod_idioma

In [14]:
def extrair_texto(imagem):

    imagem = cv2.imread(imagem)
    imagem = realcar_detalhes(imagem)
    imagem = ajustar_brilho(imagem)
    img_cinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    img_binarizada = binarizar(img_cinza)

    psm_valor = determinar_psm(img_binarizada)
    cod_idioma = detectar_idioma(img_binarizada)

    texto = pt.image_to_string(img_binarizada, lang=cod_idioma, config=f'--tessdata-dir tessdata --psm {psm_valor}')

    return img_binarizada, texto

# 4 Testes de OCR

## 4.1 Trecho de artigo

In [15]:
imagem, texto = extrair_texto(artigo)
imagem
print(texto)

Antigamente, a gestão de dados estava centrada em informações estruturadas,
seguindo o modelo rígido como de um Data Warehouse. No entanto, com o
surgimento de fontes diversas e dados não estruturados, surgiram desafios na
absorção e análise dessas informações em diferentes formatos. Mas, para lidar
com esse tipo de problema, era necessário criar soluções customizadas e
complexas, tornando o processo lento e ineficiente.

As soluções tradicionais de armazenamento e análise não estavam preparadas
para absorver, processar e extrair valores dessas informações de diferentes
formatos. Era como tentar montar um quebra-cabeça com peças que não se
encaixavam no modelo rígido preestabelecido pelo Data Warehouse. Para resolver
i8so, as empresas se viam obrigadas a recorrer a soluções customizadas e
trabalhosas, envolvendo processos complexos de pré-processamento e
transformação dos dados para torná-los adequados ao modelo tradicional.

Então, como solução à esse problema, o conceito de Data Lake

## 4.2 Trecho de código do Visual Studio Code

In [16]:
imagem, texto = extrair_texto(codigo)
imagem
print(texto)

voculary » home > @0 views.py > @ home

1
2
3
4
5
6
7
8
9

10
11
12
13
14
15
16
17
18
19
20

FRERR

26
27

from django.shortcuts import render
from django.contrib.auth.decorators import login_required

from .forms import UploadimagesForam
from .utils.extrair_texto import extrair_texto

@login_required(Login url="/login")
def home(reguest):
texto = **
imagem = **
if request.method == ‘POST":
form = UploadimagesForm(request.POST, request.FILES)
if form.is_valid():
| images = form.save(comwit=False)
images.usuario = request.user
imagem.save()
texto = extrair_texto(images.images.path)

else:

| form = UploadImagesForm()

context = {

‘form’ : form,

‘texto’: texto, # Aqui passamos o texto como contexto
}

 

 

return render(request, ‘home/index.html’, comtext)

 



## 4.3 Nota fiscal

In [17]:
imagem, texto = extrair_texto(nota_fiscal)
imagem
print(texto)

@ cobreficil

NOME FANTASIA LTDA

RUA 9 N"1006 QD .H34 LT .10 SETOR CANOAS
SAO JOSE - SC

CNP2J: 89.455.000/003-00 06/01/2022
IE: 10.457.671-2 12:12:39
IM: 08641569 CCF:120289

CDD:124857

CUPOM FISCAL

ITEM COD. DESC. VALOR
001 268 PAO SIRIOSAID 268G 3,96
002 146 QUIBE CRU 0,223KG X 27,90 5,90
003 138 HOMOS 0,2KG X 28,90 5,63

ESFIHA ABERTA 0.654 X 27,90 18,24

DINHEIRO R$ 50,00 TOTAL R$33,74

TROCO R$ 16,26
IMPOSTOS 225% R$759

T1 01T 17,00%
ND-5: 2COF4658121658H0O56874Q5
OPERADOR: MARIA DA SILVA

456HDOS7 HUKSOJAH56 UHNL9634 896QHB6GKO
BEMATECH MP-40 TH Fl ECF-IF

VER.01.002 ECF/002 L1.001

QQQQQQQOETUTITU 06/01/2022 12:12:28

FAB: BEO9912789753009677

